In [4]:
import cupy as cp 
import numpy as np
from scipy.sparse import coo_matrix
import scipy
import time
import pandas as pd
import scipy.io as sio


import os 

In [221]:
# Construct the table of reference to map N/density to the file names.
table = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])
results = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])
table['4000']['0.0001'] = 'bcsstm24.mat'
table['4000']['0.001'] = 'c-24.mat'
table['4000']['0.01'] = 'crystk01.mat'
table['4000']['0.1'] = 'heart1.mat'
table['8000']['0.0001'] = 'bcsstm38.mat'
table['8000']['0.001'] = 'c-39.mat'
table['8000']['0.01'] = 'msc10848.mat'
table['8000']['0.1'] = 'human_gene2.mat' # previously 'TSC_OPF_1047.mat' 
table['40000']['0.0001'] = 'cond-mat-2005.mat'
table['40000']['0.001'] = 'bbmat.mat'
table['40000']['0.01'] = 'TSOPF_RS_b2383_c1.mat'
table['40000']['0.1'] = '' # Does not exist.
table['80000']['0.0001'] = 'net4-1.mat'
table['80000']['0.001'] = 'consph.mat'
table['80000']['0.01'] = '' # Does not exist
table['80000']['0.1'] = '' # Does not exist
table['160000']['0.0001'] = 'para-4.mat'
table['160000']['0.001'] = 'pkustk14.mat'
table['160000']['0.01'] = '' # Does not exist
table['160000']['0.1'] = '' # Does not exist
table

,4000,8000,40000,80000,160000
0.0001,bcsstm24.mat,bcsstm38.mat,cond-mat-2005.mat,net4-1.mat,para-4.mat
0.001,c-24.mat,c-39.mat,bbmat.mat,consph.mat,pkustk14.mat
0.01,crystk01.mat,msc10848.mat,TSOPF_RS_b2383_c1.mat,,
0.1,heart1.mat,human_gene2.mat,,,


In [222]:
# Handle all the edge cases for all files
def load_matrix(matrix_name):
    data = sio.loadmat(matrix_name)
    P = data['Problem']
    zeros = {'net4-1.mat'}
    twos = {'consph.mat','human_gene2.mat','TSOPF_RS_b2383_c1.mat','cond-mat-2005.mat', 'para-4.mat'}
    fours = {'c-24.mat','c-39.mat'}
    if matrix_name in zeros:
        x = P[0][0][0]
    elif matrix_name in twos:
        x = P[0][0][2]
    elif matrix_name in fours:
        x = P[0][0][4]
    else:
        x = P[0][0][1]
    return x

def multiply_and_time(matrix_name, N, p, results):
    if not matrix_name:
        return
    x = load_matrix(matrix_name)
    x_gpu = cp.sparse.csr_matrix(x) # Convert to Cupy GPU CSR matrix.
    nonzeros = x_gpu.count_nonzero()
    density = nonzeros / (x_gpu.shape[0] * x_gpu.shape[1])
    print('\tShape of {0} is {1} with density={2}'.format(matrix_name, x_gpu.shape, density))
    start = time.time()
    x_gpu.dot(x_gpu.T)
    end = time.time()
    print("\tTime for {0} = {1}".format(matrix_name, end-start))
    results[N][p] = end-start
    return


def multiply_all(table):
    for N in ['4000', '8000', '40000', '80000', '160000']:
        for p in ['0.0001', '0.001', '0.01', '0.1']:
            print("For N={0}, p={1}".format(N,p))
            multiply_and_time(table[N][p], N, p, results)
            

In [223]:
multiply_all(table)
print("Done!")

For N=4000, p=0.0001
	Shape of bcsstm24.mat is (3562, 3562) with density=0.00028074115665356543
	Time for bcsstm24.mat = 0.010546207427978516
For N=4000, p=0.001
	Shape of c-24.mat is (4119, 4119) with density=0.002104129508142708
	Time for c-24.mat = 0.04594612121582031
For N=4000, p=0.01
	Shape of crystk01.mat is (4875, 4875) with density=0.01329192899408284
	Time for crystk01.mat = 0.09349727630615234
For N=4000, p=0.1
	Shape of heart1.mat is (3557, 3557) with density=0.10949175913309958
	Time for heart1.mat = 1.2327358722686768
For N=8000, p=0.0001
	Shape of bcsstm38.mat is (8032, 8032) with density=0.00016252532201869812
	Time for bcsstm38.mat = 0.01033473014831543
For N=8000, p=0.001
	Shape of c-39.mat is (9271, 9271) with density=0.0013564286839588878
	Time for c-39.mat = 0.10832405090332031
For N=8000, p=0.01
	Shape of msc10848.mat is (10848, 10848) with density=0.010450249519234952
	Time for msc10848.mat = 0.5044164657592773
For N=8000, p=0.1
	Shape of human_gene2.mat is (1434

In [227]:
results

,4000,8000,40000,80000,160000
0.0001,0.0105462,0.0103347,0.337611,8.66055,2.80819
0.001,0.0459461,0.108324,0.352304,1.28031,4.71171
0.01,0.0934973,0.504416,145.17,NaN,NaN
0.1,1.23274,72.6402,NaN,NaN,NaN


In [126]:
def multiply_and_time(matrix_name):
    data = sio.loadmat(matrix_name)
    P = data['Problem']
    if matrix_name == 'SiO2.mat':
        x = P[0][0][2] # For some reason, this file is stored differently.
    elif matrix_name == 'net4-1.mat':
        x = P[0][0][0]
    else:
        x = P[0][0][1]
    x_gpu = cp.sparse.csr_matrix(x) # Convert to Cupy GPU CSR matrix.
    nonzeros = x_gpu.count_nonzero()
    density = nonzeros / (x_gpu.shape[0] * x_gpu.shape[1])
    print('\tShape of {0} is {1} with density={2}'.format(matrix_name, x_gpu.shape, density))
    start = time.time()
    x_gpu.dot(x_gpu.T)
    end = time.time()
    print("Time for {0} = {1}".format(matrix_name, end-start))
    return

In [127]:
# Get all files:
matrices = [file for file in os.listdir('./') if '.mat' in file]
print("Running multiply on ", matrices)
for matrix_name in matrices:
    multiply_and_time(matrix_name)

Running multiply on  ['net4-1.mat', 'msc10848.mat']
	Shape of net4-1.mat is (88343, 88343) with density=0.00031286200139439834
Time for net4-1.mat = 8.725044250488281
	Shape of msc10848.mat is (10848, 10848) with density=0.010450249519234952
Time for msc10848.mat = 0.5050654411315918


In [220]:
matrix1 = 'pkustk14.mat'
matrix1_data = sio.loadmat(matrix1)
P = matrix1_data['Problem']
x = P[0][0][1]

x_gpu = cp.sparse.csr_matrix(x)
start = time.time()
x_gpu.dot(x_gpu.T)
end = time.time()
print("Time for {0}".format(matrix1))
print(end-start)
x_gpu

Time for pkustk14.mat
4.782956838607788


In [218]:
matrix1_data['Problem'][0][0][2]

<153226x153226 sparse matrix of type '<class 'numpy.float64'>'
	with 2930882 stored elements in Compressed Sparse Column format>

In [163]:
x

<8184x8184 sparse matrix of type '<class 'numpy.complex128'>'
	with 127762 stored elements in Compressed Sparse Column format>

In [83]:
x.count_nonzero()

array(11284032)

In [77]:
matrix1_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Sep  6 10:02:30 2008',
 '__version__': '1.0',
 '__globals__': [],
 'Problem': array([[ (array(['Mittelmann/rail4284'],
       dtype='<U19'), array(['Italian railways (H. Mittelmann test set)'],
       dtype='<U41'), array([[1658]], dtype=uint16), array(['linear programming problem'],
       dtype='<U26'), array(['2005'],
       dtype='<U4'), array(['P. Nobili'],
       dtype='<U9'), array(['J. Beasley'],
       dtype='<U10'), <4284x1096894 sparse matrix of type '<class 'numpy.float64'>'
 	with 11284032 stored elements in Compressed Sparse Column format>, array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]], dtype=uint8), array([[ (array([[0],
        [0],
        [0],
        ..., 
        [2],
        [2],
        [2]], dtype=uint8), array([[0],
        [0],
        [0],
        ..., 
        [0],
        [0],
        [0]], dtype=uint8), array([[ inf],
        [ inf],
       

,4000,8000,20000,80000,160000
0.0001,,,,,
0.001,,,,,
0.01,,msc10848.mat,,,
0.1,,,,,
